In [2]:
# Import Libraries 
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

In [5]:
# Import Dataset 
import kagglehub 

path = kagglehub.dataset_download("adarshsng/lending-club-loan-data-csv")

print("Path to dataset files:", path)

100%|██████████| 339M/339M [00:38<00:00, 9.13MB/s] 

Extracting files...


Path to dataset files: /Users/davidecosta/.cache/kagglehub/datasets/adarshsng/lending-club-loan-data-csv/versions/1


In [7]:
# List files in the directory to find the correct file name
files = os.listdir(path)
print("Files in directory:", files)

# Assuming the correct file is the first CSV file in the directory
csv_file = os.path.join(path, [file for file in files if file.endswith('.csv')][0])

# Load data into a pandas DataFrame
df = pd.read_csv(csv_file)

# Display the first few rows
df.head()

Files in directory: ['loan.csv', 'LCDataDictionary.xlsx']


/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_23080/1057770976.py:9: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
